<a href="https://colab.research.google.com/github/douglasmmachado/MedicineConsumption/blob/main/notebooks/timeseries/unified_approach/5_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 - Forecasting and prediction



---



---



In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math as m

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,  mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV


df_agg_clusters_4_url = "https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/time_series/clustered/df_clustered_4.csv"
df_agg_clusters_8_url = "https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/time_series/clustered/df_clustered_8.csv"
df_agg_clusters_12_url = "https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/time_series/clustered/df_clustered_12.csv"
df_agg_clusters_18_url = "https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/time_series/clustered/df_clustered_18.csv"

df_agg_clusters_4 = pd.read_csv(df_agg_clusters_4_url)
df_agg_clusters_8 = pd.read_csv(df_agg_clusters_8_url)
df_agg_clusters_12 = pd.read_csv(df_agg_clusters_12_url)
df_agg_clusters_18 = pd.read_csv(df_agg_clusters_18_url)

medicines = ['3400892088310','3400892075761','3400892203645',
             '3400892065366','3400892052120','3400891996128',
             '3400893826706','3400893736135','3400893875490',
             '3400890837149','3400891235203','3400891225037',
             '3400891191226','3400892729589','3400892745848',
             '3400892697789','3400892761527','3400893022634',
             '3400892761695','3400892669236','3400892508566']

In [26]:
df_agg_clusters_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             2479 non-null   int64  
 1   HOSPI_CODE_UCD    2479 non-null   int64  
 2   QUANTITY          2479 non-null   float64
 3   N_UFS             2479 non-null   float64
 4   DATE              2479 non-null   object 
 5   N_ETB             2479 non-null   float64
 6   POPULATION        2479 non-null   float64
 7   P_MEDICAL         2479 non-null   float64
 8   PN_MEDICAL        2479 non-null   float64
 9   LIT_HC            2479 non-null   float64
 10  LIT_HP            2479 non-null   float64
 11  SEJ_MCO           2479 non-null   float64
 12  SEJ_HAD           2479 non-null   float64
 13  SEJ_PSY           2479 non-null   float64
 14  SEJ_SSR           2479 non-null   float64
 15  SEJ_SLD           2479 non-null   float64
 16  TREND             2479 non-null   float64


## 5.1 - New database composition based on clusters

## 5.2 - Building forecasting models based on clusters

In [19]:
def plot_pred(y_pred, y_test, medicine):
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))

    # Scatter plot for y_pred
    axes[0].scatter(np.arange(len(y_pred)), y_pred, marker='o', label='Predicted', color='blue')

    # Scatter plot for y_test
    axes[0].scatter(np.arange(len(y_test)), y_test, marker='x', label='Actual', color='red')

    # Set axes labels and title
    axes[0].set_xlabel('Test samples')
    axes[0].set_ylabel('Quantity')
    axes[0].set_title(f'Predicted vs Actual: {medicine}')
    axes[0].legend()

    epsilon = 0.001
    mape_array = np.abs(y_test - y_pred) / np.maximum(epsilon, np.abs(y_test))

    # Stem plot for MAPE
    stem = axes[1].stem(np.arange(len(y_pred)), mape_array, markerfmt='bo', linefmt='b-', basefmt='r-', label='MAPE')
    axes[1].set_xlabel('Test samples')
    axes[1].set_ylabel('MAPE')
    axes[1].set_title(f'MAPE for: {medicine}')
    axes[1].set_ylim([0, 1])

    mape_target = 0.3
    axes[1].axhline(y=mape_target, color='g', linestyle='--', label=f'Target MAPE ({mape_target:.2f})')

    # Adjust layout
    plt.tight_layout()

    # Add a legend
    handles, labels = axes[1].get_legend_handles_labels()
    axes[1].legend(handles=handles, labels=labels, loc='best')

    # Adjust width of subplots and margins
    fig.subplots_adjust(wspace=0.4, left=0.1, right=0.9)

    plt.show()

In [28]:
def test_1_baseline(df, medicine, df_scores):
  print()
  print(100*'-')
  print('Medicine:' + str(medicine))

  X = df[df['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY', 'HOSPI_CODE_UCD',
                                                 'DATE', 'TREND', 'SEASONAL', 'RESID'], axis=1).values

  y = df[df['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].values

  if m.ceil(len(X) * 0.1) == 1:
    print('Dataset too small')
    test_size = 2
  else:
    test_size = 0.2

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size = test_size,
                                                      random_state = 42)
  print(f'Size of data set: {len(X)}')
  print(f'Size of training set: {len(X_train)}')
  print(f'Size of test set: {len(X_test)}')

  # Define the parameter distributions for RandomizedSearchCV
  param_grid = {
      'max_depth': np.arange(2, 8, 1),
      'n_estimators': np.arange(2, max(int(m.ceil(len(X_train)*0.1)),3), 1),
      'max_features': ['sqrt', 1, 2]
  }
  depth_len = param_grid['max_depth'].size
  estimators_len = param_grid['n_estimators'].size

  print(f'Size of grid search: {depth_len * estimators_len}')

  # Create the RandomizedSearchCV object
  grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                             param_grid=param_grid,
                             scoring = 'neg_mean_absolute_percentage_error',
                             cv = 5,
                             n_jobs = -1)

  ''' >3 : the fold and candidate parameter indexes
      are also displayed together with the starting time of the computation.
  '''
  # Fit the RandomizedSearchCV object to the data
  grid_search.fit(X_train, y_train)

  # Get the best estimator
  best_estimator = grid_search.best_estimator_

  # Make predictions using the best estimator
  y_pred = best_estimator.predict(X_test)

  # Calculate R^2 score
  r2 = r2_score(y_test, y_pred)

  # Calculate MAE
  mae = mean_absolute_error(y_test, y_pred)

  # Calculate MAPE
  mape = mean_absolute_percentage_error(y_test, y_pred)

  # Calculate RMSE
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Print the best parameters, best score, and evaluation metrics
  print('Best Parameters: ', grid_search.best_params_)
  print('Training Score (MAPE): ', round(grid_search.best_score_, 3))
  print(10*'-' + 'Test scores' + 10*'-')
  print('R^2 Score:', round(r2, 3))
  print('MAE:', round(mae, 3))
  print('MAPE:', round(mape, 3))
  print('RMSE:', round(rmse, 3))
  print()


  # Create the new row as a DataFrame
  new_row = pd.DataFrame({'HOSPI_CODE_UCD': ['CODE_UCD_'+str(medicine)],
                          'R2': [r2],
                          'RMSE': [rmse],
                          'MAE': [mae],
                          'MAPE': [mape]})

  # Append the new row to the DataFrame
  df_scores = pd.concat([df_scores, new_row], ignore_index=True)

  # plot pred x test
  plot_pred(y_pred, y_test, medicine)
  print()
  plt.close()



  # Return the updated DataFrame
  return df_scores


In [29]:
df_prediction_scores = pd.DataFrame(columns=['HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in medicines:
  df_prediction_scores = test_1_baseline(df_agg_clusters_4.drop(['CLUSTER'], axis = 1), medicine, df_prediction_scores)

df_prediction_scores


----------------------------------------------------------------------------------------------------
Medicine:3400892088310


ValueError: ignored

In [ ]:
def test_2_clustering(df, df_scores, medicines):

  for cluster in df.CLUSTER.unique():
    print()
    print(100*'-')
    print(f'Cluster: {cluster}')

    # Perform the train-test split with shuffled samples
    X = df[df['CLUSTER'] == cluster].drop(['DATE', 'QUANTITY', 'CLUSTER', 'QUANTITY_MA', 'TREND', 'SEASONAL', 'RESID'], axis=1).copy().values
    y = df[df['CLUSTER'] == cluster]['QUANTITY'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, shuffle = True)
    print(f'Size of data set: {len(X_train) + len(X_test)}')
    print(f'Size of training set: {len(X_train)}')
    print(f'Size of test set: {len(X_test)}')

    df_test = pd.DataFrame(X_test, columns = df.drop(['DATE', 'QUANTITY', 'CLUSTER', 'QUANTITY_MA', 'TREND', 'SEASONAL', 'RESID'], axis=1).copy().columns)
    df_test['QUANTITY'] = y_test


    # Define the parameter distributions for RandomizedSearchCV
    param_grid = {
        'max_depth': np.arange(2, 8, 1),
        'n_estimators': np.arange(2, max(int(m.ceil(len(X_train)*0.1)),3), 1),
        'max_features': ['sqrt', 1, 2]
    }

    depth_len = param_grid['max_depth'].size
    estimators_len = param_grid['n_estimators'].size

    print(f'Size of grid search: {depth_len * estimators_len}')

    # Create the RandomizedSearchCV object
    grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=param_grid,
                              scoring = 'neg_mean_absolute_percentage_error',
                              cv = 5,
                              n_jobs = -1)

    ''' >3 : the fold and candidate parameter indexes
        are also displayed together with the starting time of the computation.
    '''


    # Fit the RandomizedSearchCV object to the data
    grid_search.fit(X_train[:,1:], y_train)

    print("Finished training")

    # Get the best estimator
    best_estimator = grid_search.best_estimator_

    for medicine in df_test.HOSPI_CODE_UCD.unique():
      print()
      print(100*'-')
      print('Medicine:' + str(medicine))

      X_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY', 'HOSPI_CODE_UCD'], axis=1).copy().values

      scaler = StandardScaler()
      X_test_scaled = scaler.fit_transform(X_test_medicine)

      y_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].copy().values

      # Make predictions using the best estimator
      y_pred = best_estimator.predict(X_test_scaled)

      # Calculate R^2 score
      r2 = r2_score(y_test_medicine, y_pred)

      # Calculate MAE
      mae = mean_absolute_error(y_test_medicine, y_pred)

      # Calculate MAPE
      mape = mean_absolute_percentage_error(y_test_medicine, y_pred)

      # Calculate RMSE
      rmse = np.sqrt(mean_squared_error(y_test_medicine, y_pred))

      # Print the best parameters, best score, and evaluation metrics

      # Print the best parameters, best score, and evaluation metrics
      print('Best Parameters: ', grid_search.best_params_)
      print('Training Score (MAPE): ', round(grid_search.best_score_, 3))
      print(10*'-' + 'Test scores' + 10*'-')
      print('R^2 Score:', round(r2, 3))
      print('MAE:', round(mae, 3))
      print('MAPE:', round(mape, 3))
      print('RMSE:', round(rmse, 3))
      print()


      # Create the new row as a DataFrame
      new_row = pd.DataFrame({'CLUSTER': [cluster],
                              'HOSPI_CODE_UCD': ['CODE_UCD_'+str(int(medicine))],
                              'R2': [r2],
                              'RMSE': [rmse],
                              'MAE': [mae],
                              'MAPE': [mape]})

      # Append the new row to the DataFrame
      df_scores = pd.concat([df_scores, new_row], ignore_index=True)

      plot_pred(y_pred, y_test_medicine, medicine)
      print()


  # Return the updated DataFrame
  return df_scores

In [ ]:
'''df_prediction_scores_agg = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_agg = test_2_clustering(df_agg_clusters, df_prediction_scores_agg, medicines)

df_prediction_scores_agg'''

"df_prediction_scores_agg = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])\n\ndf_prediction_scores_agg = test_2_clustering(df_agg_clusters, df_prediction_scores_agg, medicines)\n\ndf_prediction_scores_agg"

In [ ]:
'''df_prediction_scores_pca = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_pca = test_2_clustering(df_clustered_pca, df_prediction_scores_pca, medicines)

df_prediction_scores_pca'''

"df_prediction_scores_pca = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])\n\ndf_prediction_scores_pca = test_2_clustering(df_clustered_pca, df_prediction_scores_pca, medicines)\n\ndf_prediction_scores_pca"